# 第19章：Async & Concurrency（异步与并发）

## 学习目标

本章将学习：
1. 异步编程的核心概念
2. `ainvoke()`、`astream()`、`abatch()` 异步方法
3. 并发执行多个任务
4. 异步与同步的性能对比
5. asyncio基础知识
6. 并发控制和优化
7. 实战：高并发API服务
8. 最佳实践和常见陷阱

---

## 为什么需要异步编程？

### 同步 vs 异步

#### 同步执行（串行）
```
任务1: ████████ (2秒)
任务2:         ████████ (2秒)  
任务3:                 ████████ (2秒)
总时间: 6秒
```

#### 异步执行（并发）
```
任务1: ████████ (2秒)
任务2: ████████ (2秒)  
任务3: ████████ (2秒)
总时间: 2秒（并发执行）
```

### 核心优势

| 维度 | 同步 | 异步 |
|------|------|------|
| **执行方式** | 串行，一个接一个 | 并发，多个同时执行 |
| **I/O等待** | 阻塞，浪费时间 | 非阻塞，利用等待时间 |
| **吞吐量** | 低 | 高（3-10倍提升） |
| **适用场景** | CPU密集型 | I/O密集型（API调用） |
| **复杂度** | 简单 | 相对复杂 |

### LangChain中的I/O密集型任务

- ✅ **LLM API调用**：等待模型响应（主要场景）
- ✅ **向量数据库查询**：网络I/O
- ✅ **外部API调用**：工具执行
- ✅ **文档加载**：文件I/O

**关键**：LLM API调用通常需要1-5秒，是典型的I/O密集型任务，使用异步可以大幅提升性能。

---

## 核心概念

### 1. Python asyncio基础

```python
import asyncio

# 定义异步函数
async def fetch_data():
    await asyncio.sleep(1)  # 模拟I/O等待
    return "data"

# 运行异步函数
result = await fetch_data()  # 在Jupyter中可以直接用await
# 或
result = asyncio.run(fetch_data())  # 在脚本中使用
```

**关键字**：
- `async def`：定义异步函数
- `await`：等待异步操作完成
- `asyncio.gather()`：并发执行多个异步任务

### 2. LangChain的异步方法

所有主要方法都有异步版本（加`a`前缀）：

| 同步方法 | 异步方法 | 说明 |
|---------|---------|------|
| `invoke()` | `ainvoke()` | 单次调用 |
| `stream()` | `astream()` | 流式输出 |
| `batch()` | `abatch()` | 批量处理 |

### 3. 并发 vs 并行

- **并发（Concurrency）**：同时处理多个任务（通过任务切换）
  - 适合I/O密集型
  - 单核CPU也能实现
  
- **并行（Parallelism）**：同时执行多个任务（真正的多核执行）
  - 适合CPU密集型
  - 需要多核CPU

**本章重点**：并发（异步I/O）


In [6]:
# 环境配置
import os
import sys
import time
import asyncio

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

print("环境配置完成！")
print(f"模型: {model.model_name}")

环境配置完成！
模型: gpt-4.1-mini


## 1. ainvoke() - 异步调用

### 核心价值

`ainvoke()`是`invoke()`的异步版本，适合需要并发执行多个LLM调用的场景。

### 使用方法

```python
# 同步调用
response = model.invoke("问题")

# 异步调用
response = await model.ainvoke("问题")
```

### 典型场景

- ✅ 同时调用多个不同的LLM
- ✅ 批量处理用户请求
- ✅ 并发执行独立的Agent任务

In [7]:
print("【演示1：同步 vs 异步性能对比】\n")

# 定义6个问题
questions = [
    "用一句话介绍Python",
    "用一句话介绍JavaScript", 
    "用一句话介绍Rust",
    "用一句话介绍Go",
    "用一句话介绍Java",
    "用一句话介绍C++",
]

# 方式1：同步调用（串行）
print("=== 方式1：同步调用（串行） ===")
start = time.time()

for q in questions:
    response = model.invoke(q)
    print(f"  问题: {q[:20]}... → 回答: {response.content[:40]}...")

sync_time = time.time() - start
print(f"同步总耗时: {sync_time:.2f}秒\n")

# 方式2：异步调用（并发）
print("=== 方式2：异步调用（并发） ===")
start = time.time()

async def async_invoke_all():
    # asyncio.gather()并发执行多个异步任务
    tasks = [model.ainvoke(q) for q in questions]
    responses = await asyncio.gather(*tasks)
    
    for q, response in zip(questions, responses):
        print(f"  问题: {q[:20]}... → 回答: {response.content[:40]}...")
    
    return responses

# 在Jupyter中可以直接await
responses = await async_invoke_all()
async_time = time.time() - start

print(f"异步总耗时: {async_time:.2f}秒")
print(f"\n✓ 性能提升: {sync_time/async_time:.1f}x 加速")

【演示1：同步 vs 异步性能对比】

=== 方式1：同步调用（串行） ===
  问题: 用一句话介绍Python... → 回答: Python是一种简洁易学、功能强大的高级编程语言，广泛应用于数据分析、人工智能...
  问题: 用一句话介绍JavaScript... → 回答: JavaScript是一种广泛用于网页开发的动态脚本语言，能够实现网页的交互效果...
  问题: 用一句话介绍Rust... → 回答: Rust是一种注重安全性、并发性和性能的系统编程语言，适合构建高效可靠的软件。...
  问题: 用一句话介绍Go... → 回答: Go是一种由Google开发的开源编程语言，具有简洁、高效、并发友好的特点，适用...
  问题: 用一句话介绍Java... → 回答: Java是一种跨平台的面向对象编程语言，广泛用于开发企业级应用、移动应用和大型系...
  问题: 用一句话介绍C++... → 回答: C++是一种支持面向对象编程的高性能通用编程语言，广泛用于系统软件、游戏开发和嵌...
同步总耗时: 11.97秒

=== 方式2：异步调用（并发） ===
  问题: 用一句话介绍Python... → 回答: Python是一种简洁易学、功能强大的高级编程语言，广泛应用于数据分析、人工智能...
  问题: 用一句话介绍JavaScript... → 回答: JavaScript是一种广泛用于网页开发的动态脚本语言，能够实现网页的交互效果...
  问题: 用一句话介绍Rust... → 回答: Rust是一种注重安全性、并发性和性能的系统编程语言，适合构建高效可靠的软件。...
  问题: 用一句话介绍Go... → 回答: Go是一种由Google开发的开源编程语言，具有简洁、高效、并发友好的特点，适用...
  问题: 用一句话介绍Java... → 回答: Java是一种面向对象、跨平台的高级编程语言，广泛用于开发企业级应用、移动应用和...
  问题: 用一句话介绍C++... → 回答: C++是一种支持面向对象、泛型编程和低级内存操作的高性能通用编程语言。...
异步总耗时: 3.15秒

✓ 性能提升: 3.8x 加速


## 2. astream() - 异步流式输出

### 核心价值

`astream()`结合了异步和流式输出的优势，适合需要实时反馈的并发场景。

### 使用场景

- ✅ 多个用户同时聊天（WebSocket）
- ✅ 并发处理多个流式请求
- ✅ 实时显示多个Agent的输出

### 语法

```python
async for chunk in model.astream("问题"):
    print(chunk.content, end="", flush=True)
```

In [8]:
print("【演示2：并发流式输出】\n")

async def stream_one_question(question: str, label: str):
    """流式处理一个问题"""
    print(f"{label}: ", end="", flush=True)
    
    full_response = ""
    async for chunk in model.astream(question):
        if chunk.content:
            full_response += chunk.content
            print(chunk.content, end="", flush=True)
    
    print()  # 换行
    return full_response

# 并发执行3个流式任务
async def concurrent_streaming():
    tasks = [
        stream_one_question("Python的主要优点？", "问题1"),
        stream_one_question("异步编程的核心概念？", "问题2"),
        stream_one_question("LangChain是什么？", "问题3"),
    ]
    
    # 注意：gather会等所有任务完成，但它们是并发执行的
    results = await asyncio.gather(*tasks)
    return results

print("开始并发流式输出（3个问题同时进行）：\n")
start = time.time()
results = await concurrent_streaming()
elapsed = time.time() - start

print(f"\n✓ 并发流式输出完成，耗时: {elapsed:.2f}秒")
print("  （如果串行执行，耗时会是3倍）")

【演示2：并发流式输出】

开始并发流式输出（3个问题同时进行）：

问题1: 问题2: 问题3: 异步编程的核心概念主要Python的主要优点LangChain包括：

1. **语法简洁易读**  
   Python的语法设计非常简洁，包括以下几个方面：

1. **非阻塞（Non-blocking）**接近自然语言，代码可读性强，适合初学者和  
   异步编程允许程序在等待某些操作完成（如I 是一个用于构建基于大型语言模型（LLM）的应用程序的开源框架。它提供了一套工具和组件，帮助开发者更方便快速开发。

2. **丰富的标准库和第三方库**  
   Python拥有庞大的标准库，涵盖文件操作、网络、数据库、正则表达式等多种功能。同时，第三方库生态丰富，如NumPy、Pandas、TensorFlow、Django等，支持数据分析、机器学习、Web开发等多领域应用。

3. **跨平台性强**  
   Python可以运行在Windows、Linux、macOS等多种操作系统上，代码具有良好的移植性。

4. **支持多种编程范式**  
   Python支持面向对象编程地将语言模型集成到各种应用中，实现复杂的自然语言处理任务和工作流自动化。

具体来说，LangChain/O操作、网络请求）时，不阻塞当前线程，可以、过程式编程和函数式编程，灵活继续执行其他任务，提高资源利用率和程序响应性。

2. **回调（Callback）**  
   异步操作完成后，通过回调函数通知程序结果适应不同的开发需求。

5. **动态类型和自动内存管理**  
   Python。回调函数是在异步操作完成时被调用的函数，用于处理结果 的核心功能包括：

1. **链（Chains）**：将多个语言模型调用和其他计算步骤是动态类型语言，变量无需声明类型，简化开发过程。内置垃圾回收机制，自动管理内存，串联起来，形成复杂的处理流程。例如，先用模型生成文本，再用减少内存泄漏风险。

另一个模型进行摘要或分析。

2. **代理（Agents）**：6. **强大的社区支持**  
  允许模型根据上下文动态决定调用哪些工具或执行哪些操作，实现更智能或错误。

3. **事件循环（Event Loop）** Python拥有活跃的开发者社区，丰富的文档和教程，遇到问题时容易

## 3. abatch() - 异步批处理

### 核心价值

`abatch()`并发处理批量输入，比循环调用`ainvoke()`更高效。

### 对比

```python
# 方式1：循环ainvoke（不推荐）
results = []
for input in inputs:
    result = await model.ainvoke(input)
    results.append(result)

# 方式2：使用abatch（推荐）
results = await model.abatch(inputs)
```

**优势**：
- ✅ 自动并发控制
- ✅ 更简洁的代码
- ✅ 统一的错误处理

In [9]:
print("【演示3：批量处理】\n")

# 准备10个不同的翻译任务
translation_tasks = [
    "将'Hello'翻译成中文",
    "将'Thank you'翻译成中文",
    "将'Good morning'翻译成中文",
    "将'How are you'翻译成中文",
    "将'Goodbye'翻译成中文",
]

# 方式1：循环ainvoke
print("=== 方式1：循环ainvoke ===")
start = time.time()

results1 = []
for task in translation_tasks:
    result = await model.ainvoke(task)
    results1.append(result)

time1 = time.time() - start
print(f"耗时: {time1:.2f}秒")
print(f"示例结果: {results1[0].content}\n")

# 方式2：使用abatch（更高效）
print("=== 方式2：使用abatch（推荐） ===")
start = time.time()

results2 = await model.abatch(translation_tasks)

time2 = time.time() - start
print(f"耗时: {time2:.2f}秒")
print(f"示例结果: {results2[0].content}")

print(f"\n✓ abatch比循环ainvoke快 {time1/time2:.1f}x")
print("✓ abatch自动管理并发，代码更简洁")

【演示3：批量处理】

=== 方式1：循环ainvoke ===
耗时: 9.39秒
示例结果: “Hello” 翻译成中文是 “你好”。

=== 方式2：使用abatch（推荐） ===
耗时: 2.82秒
示例结果: “Hello” 翻译成中文是 “你好”。

✓ abatch比循环ainvoke快 3.3x
✓ abatch自动管理并发，代码更简洁


## 4. Agent异步执行

### 核心价值

Agent也支持异步方法，适合并发处理多个Agent任务。

### 异步Agent方法

| 同步 | 异步 | 说明 |
|------|------|------|
| `agent.invoke()` | `agent.ainvoke()` | 执行Agent |
| `agent.stream()` | `agent.astream()` | 流式执行 |
| `agent.batch()` | `agent.abatch()` | 批量执行 |

### 典型场景

- ✅ 多用户并发访问Agent
- ✅ 并发执行多个独立的Agent任务
- ✅ 子Agent并发执行（Multi-Agent）

In [10]:
print("【演示4：并发Agent执行】\n")

# 创建一个简单的Agent
@tool
def get_weather(city: str) -> str:
    """获取城市天气"""
    return f"{city}今天晴天，25°C"

agent = create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="你是天气助手"
)

# 准备多个用户请求
user_requests = [
    {"messages": [{"role": "user", "content": "北京天气"}]},
    {"messages": [{"role": "user", "content": "上海天气"}]},
    {"messages": [{"role": "user", "content": "深圳天气"}]},
]

# 方式1：同步串行执行
print("=== 方式1：同步串行执行 ===")
start = time.time()

for req in user_requests:
    response = agent.invoke(req)
    city = req["messages"][0]["content"][:4]
    print(f"  {city}: {response['messages'][-1].content[:30]}...")

sync_time = time.time() - start
print(f"耗时: {sync_time:.2f}秒\n")

# 方式2：异步并发执行
print("=== 方式2：异步并发执行 ===")
start = time.time()

async def process_all_requests():
    tasks = [agent.ainvoke(req) for req in user_requests]
    responses = await asyncio.gather(*tasks)
    
    for req, response in zip(user_requests, responses):
        city = req["messages"][0]["content"][:4]
        print(f"  {city}: {response['messages'][-1].content[:30]}...")
    
    return responses

responses = await process_all_requests()
async_time = time.time() - start

print(f"耗时: {async_time:.2f}秒")
print(f"\n✓ 异步并发比同步串行快 {sync_time/async_time:.1f}x")
print("✓ 适合多用户并发场景")

【演示4：并发Agent执行】

=== 方式1：同步串行执行 ===
  北京天气: 北京今天是晴天，气温约为25°C。需要了解其他城市的天气吗？...
  上海天气: 上海今天是晴天，气温约为25°C。需要了解其他城市的天气吗？...
  深圳天气: 深圳今天是晴天，气温约为25°C。需要了解其他城市的天气吗？...
耗时: 17.06秒

=== 方式2：异步并发执行 ===
  北京天气: 北京今天是晴天，气温约为25°C。需要了解其他城市的天气吗？...
  上海天气: 上海今天是晴天，气温大约25°C。需要了解其他城市的天气吗？...
  深圳天气: 深圳今天是晴天，气温约为25°C。需要了解其他城市的天气吗？...
耗时: 5.25秒

✓ 异步并发比同步串行快 3.2x
✓ 适合多用户并发场景


## 5. 并发控制 - 限制并发数量

### 为什么需要限制？

并发太多会导致：
- ❌ API速率限制（Rate Limit）
- ❌ 内存占用过高
- ❌ 连接数超限

### 解决方案：Semaphore

```python
import asyncio

# 限制同时最多5个并发任务
semaphore = asyncio.Semaphore(5)

async def controlled_task(data):
    async with semaphore:  # 自动控制并发数
        return await process(data)
```

### 推荐并发数

| API类型 | 建议并发数 | 原因 |
|---------|-----------|------|
| OpenAI | 10-50 | 有速率限制 |
| Claude | 10-50 | 有速率限制 |
| 自建模型 | 按服务器能力 | 受硬件限制 |
| 向量数据库 | 100+ | 通常支持高并发 |

In [15]:
print("【演示5：并发控制】\n")

# 模拟20个任务，但限制同时只能执行3个
tasks_data = [f"任务{i+1}" for i in range(20)]

# 创建Semaphore限制并发数
semaphore = asyncio.Semaphore(3)  # 最多3个并发

async def controlled_llm_call(data: str):
    """受控的LLM调用"""
    async with semaphore:  # 进入时获取许可，退出时释放
        print(f"  开始执行: {data}")
        # 模拟LLM调用
        result = await model.ainvoke(f"简单回复'{data}'")
        print(f"  完成: {data}")
        return result

print("开始执行20个任务（限制并发数=3）\n")
start = time.time()

# 并发执行所有任务，但Semaphore会自动限制并发数
tasks = [controlled_llm_call(data) for data in tasks_data[:20]]  # 只执行5个演示
results = await asyncio.gather(*tasks)

elapsed = time.time() - start

print(f"\n✓ 执行完成，耗时: {elapsed:.2f}秒")
print("✓ Semaphore确保不会超过并发限制")
print("✓ 避免触发API速率限制")

【演示5：并发控制】

开始执行20个任务（限制并发数=3）

  开始执行: 任务1
  开始执行: 任务2
  开始执行: 任务3
  完成: 任务2
  开始执行: 任务4
  完成: 任务1
  开始执行: 任务5
  完成: 任务3
  开始执行: 任务6
  完成: 任务4
  开始执行: 任务7
  完成: 任务5
  开始执行: 任务8
  完成: 任务6
  开始执行: 任务9
  完成: 任务7
  开始执行: 任务10
  完成: 任务8
  开始执行: 任务11
  完成: 任务9
  开始执行: 任务12
  完成: 任务10
  开始执行: 任务13
  完成: 任务11
  开始执行: 任务14
  完成: 任务12
  开始执行: 任务15
  完成: 任务14
  开始执行: 任务16
  完成: 任务15
  开始执行: 任务17
  完成: 任务13
  开始执行: 任务18
  完成: 任务17
  开始执行: 任务19
  完成: 任务16
  开始执行: 任务20
  完成: 任务18
  完成: 任务19
  完成: 任务20

✓ 执行完成，耗时: 12.62秒
✓ Semaphore确保不会超过并发限制
✓ 避免触发API速率限制


## 6. 实战案例：高并发API服务

### 场景

构建一个Web API服务，同时处理多个用户请求。

### 架构

```
多个用户 → FastAPI → 异步Agent → LLM API
   ↓           ↓         ↓
 并发请求   异步处理   并发调用
```

### 核心代码模式

```python
from fastapi import FastAPI
import asyncio

app = FastAPI()

@app.post("/chat")
async def chat_endpoint(user_message: str):
    # 异步处理用户请求
    response = await agent.ainvoke({
        "messages": [{"role": "user", "content": user_message}]
    })
    return response["messages"][-1].content
```

**关键点**：
- ✅ 使用FastAPI的`async def`
- ✅ Agent使用`ainvoke()`
- ✅ 自动支持高并发

In [ ]:
print("【演示6：模拟高并发API - 批量处理模式】\n")

# 模拟Web API处理函数
async def handle_user_request(user_id: int, message: str):
    """模拟处理单个用户请求"""
    print(f"  用户{user_id}: 开始处理...")
    
    # 异步调用Agent
    response = await agent.ainvoke({
        "messages": [{"role": "user", "content": message}]
    })
    
    result = response["messages"][-1].content[:40]
    print(f"  用户{user_id}: 完成 → {result}...")
    
    return {"user_id": user_id, "response": result}

# 模拟10个并发用户请求
print("模拟10个用户同时发送请求：\n")
start = time.time()

# 创建10个并发请求
concurrent_requests = [
    handle_user_request(i+1, f"问题{i+1}：介绍一个概念")
    for i in range(10)
]

# ⚠️ 注意：asyncio.gather() 会等待所有任务完成后才返回
# 这适合批量处理场景，但在真实Web API中，每个请求应该独立返回
results = await asyncio.gather(*concurrent_requests)

elapsed = time.time() - start

print(f"\n✓ 成功处理10个并发请求")
print(f"✓ 总耗时: {elapsed:.2f}秒")
print(f"✓ 平均响应时间: {elapsed/10:.2f}秒/请求")
print("\n⚠️ 说明：")
print("  - asyncio.gather() 等待所有任务完成后才返回")
print("  - 这适合批量处理场景（如批量翻译、批量分析）")
print("  - 在真实Web API中，每个用户应该独立得到响应！")

【演示6：模拟高并发API】

模拟10个用户同时发送请求：

  用户1: 开始处理...
  用户2: 开始处理...
  用户3: 开始处理...
  用户4: 开始处理...
  用户5: 开始处理...
  用户6: 开始处理...
  用户7: 开始处理...
  用户8: 开始处理...
  用户9: 开始处理...
  用户10: 开始处理...
  用户1: 完成 → 请问您想了解哪个方面的概念？比如科技、经济、文化、自然科学等？这样我可以为您提供...
  用户7: 完成 → 请问您想了解哪个领域或方面的概念？比如科技、经济、文化、教育等？这样我可以为您提...
  用户3: 完成 → 请问您想了解哪个领域或方面的概念？比如科技、经济、心理学、物理等？这样我可以为您...
  用户6: 完成 → 请问您想了解哪个领域或方面的概念？比如科技、经济、心理学、物理等。这样我可以为您...
  用户2: 完成 → 请问您想了解哪个领域或方面的概念？比如科技、经济、文化、教育等？这样我可以为您提...
  用户5: 完成 → 请问您想了解哪个领域或方面的概念？比如科技、经济、文化、教育等？这样我可以为您提...
  用户8: 完成 → 请问您想了解哪个领域或方面的概念？比如科技、经济、文化、教育等？这样我可以为您提...
  用户10: 完成 → 请问您想了解哪个领域或方面的概念？比如科技、经济、心理学、物理等？这样我可以为您...
  用户9: 完成 → 请问您想了解哪个领域或方面的概念？比如科技、经济、心理学、物理等？这样我可以为您...
  用户4: 完成 → 请问您想了解哪个领域或方面的概念？比如科技、经济、文化、教育等？这样我可以为您提...

✓ 成功处理10个并发请求
✓ 总耗时: 3.72秒
✓ 平均响应时间: 0.37秒/请求

如果是同步处理，10个请求需要串行执行，耗时会是10倍！


### 6.2 真实Web API模式 - 独立返回

在真实的Web API场景（如FastAPI、Flask）中：
- ✅ **每个用户请求独立处理**
- ✅ **谁先完成谁先得到响应**
- ✅ **不需要等待其他用户**

#### 工作机制

```
用户1请求 → FastAPI → ainvoke → 3秒完成 → 立即返回给用户1
用户2请求 → FastAPI → ainvoke → 2秒完成 → 立即返回给用户2 ✓先返回
用户3请求 → FastAPI → ainvoke → 4秒完成 → 立即返回给用户3
```

每个请求都是独立的HTTP连接，完成后立即返回，不等待其他请求。

In [18]:
print("【演示6.2：真实Web API - 独立返回模式】\n")

# 模拟不同处理时间的请求
async def handle_request_independent(user_id: int, delay: float):
    """模拟单个用户请求（独立处理）"""
    print(f"  用户{user_id}: 开始处理...")
    
    # 模拟不同的处理时间
    await asyncio.sleep(delay)
    
    # 完成后立即"返回"给用户
    print(f"  ✓ 用户{user_id}: 完成！({delay}秒) → 立即返回给用户{user_id}")
    
    return {"user_id": user_id, "response": f"响应{user_id}", "time": delay}

print("模拟3个用户请求（不同处理时间）：\n")

# 使用asyncio.create_task创建独立任务，而不是gather
# 这样每个任务完成后会立即执行后续代码
tasks = []
start = time.time()

# 用户1：需要5秒
task1 = asyncio.create_task(handle_request_independent(1, 5.0))
tasks.append(task1)

# 用户2：需要1秒（最快）
task2 = asyncio.create_task(handle_request_independent(2, 1.0))
tasks.append(task2)

# 用户3：需要3秒
task3 = asyncio.create_task(handle_request_independent(3, 3.0))
tasks.append(task3)

# 等待所有任务完成（仅用于演示）
await asyncio.gather(*tasks)

elapsed = time.time() - start

print(f"\n✓ 总耗时: {elapsed:.2f}秒")
print("  - 每个用户独立返回，不等待其他用户！")
print("\n💡 这就是真实Web API的行为：")
print("  FastAPI/Flask会为每个请求创建独立的异步任务")
print("  谁先完成谁先得到HTTP响应")

【演示6.2：真实Web API - 独立返回模式】

模拟3个用户请求（不同处理时间）：

  用户1: 开始处理...
  用户2: 开始处理...
  用户3: 开始处理...
  ✓ 用户2: 完成！(1.0秒) → 立即返回给用户2
  ✓ 用户3: 完成！(3.0秒) → 立即返回给用户3
  ✓ 用户1: 完成！(5.0秒) → 立即返回给用户1

✓ 总耗时: 5.00秒
  - 每个用户独立返回，不等待其他用户！

💡 这就是真实Web API的行为：
  FastAPI/Flask会为每个请求创建独立的异步任务
  谁先完成谁先得到HTTP响应


### 两种模式对比

| 维度 | 批量处理模式 | Web API模式 |
|------|------------|-----------|
| **代码** | `await asyncio.gather(*tasks)` | FastAPI自动处理每个请求 |
| **返回时机** | 等所有任务完成 | 各自独立返回 |
| **适用场景** | 批量翻译、批量分析 | 多用户Web服务 |
| **用户体验** | 最慢的用户决定所有人 | 快的用户先得到响应 |
| **示例** | 批量处理100篇文章 | 网站同时服务100个用户 |

#### 真实Web API代码示例

```python
from fastapi import FastAPI

app = FastAPI()

@app.post("/chat")
async def chat_endpoint(message: str):
    # 每个HTTP请求独立处理
    # 完成后立即返回给对应用户
    # 不等待其他用户的请求
    response = await agent.ainvoke({
        "messages": [{"role": "user", "content": message}]
    })
    return response["messages"][-1].content
    # ↑ return后，这个用户立即得到响应
```

**关键点**：
- FastAPI为每个HTTP请求创建独立的协程
- 每个请求完成后立即return，触发HTTP响应
- 不同用户的请求互不等待
- 这才是真正的"并发"Web服务！


## 7. 总结与最佳实践

### 核心要点

#### 1. 何时使用异步？

| 场景 | 是否使用异步 | 原因 |
|------|------------|------|
| **单次LLM调用** | ❌ 不需要 | 没有并发，异步反而增加复杂度 |
| **多个独立LLM调用** | ✅ 推荐 | 可并发，性能提升明显 |
| **Web API服务** | ✅ 必须 | 多用户并发访问 |
| **批量处理** | ✅ 推荐 | 使用`abatch()` |
| **流式输出** | 看情况 | 单用户用`stream()`，多用户用`astream()` |

#### 2. 异步方法对照表

```python
# 同步 → 异步
model.invoke()       → await model.ainvoke()
model.stream()       → async for chunk in model.astream()
model.batch()        → await model.abatch()

agent.invoke()       → await agent.ainvoke()
agent.stream()       → async for chunk in agent.astream()

# 并发执行
tasks = [task1(), task2(), task3()]
results = await asyncio.gather(*tasks)
```

#### 3. 性能提升预期

| 并发任务数 | 预期加速比 | 说明 |
|-----------|-----------|------|
| 2-5个 | 2-4x | 线性加速 |
| 10个 | 5-8x | 接近线性 |
| 50个 | 10-20x | 受API限制 |
| 100+个 | 20-30x | 需要并发控制 |

---

### 最佳实践

#### 1. 选择合适的并发数

```python
# ✅ 好的做法：根据API限制设置
semaphore = asyncio.Semaphore(20)  # OpenAI建议20-50

async def safe_call(data):
    async with semaphore:
        return await model.ainvoke(data)

# ❌ 坏的做法：无限制并发
tasks = [model.ainvoke(data) for data in large_dataset]  # 可能触发速率限制
```

#### 2. 错误处理

```python
# ✅ 好的做法：捕获异常
async def safe_ainvoke(data):
    try:
        return await model.ainvoke(data)
    except Exception as e:
        print(f"Error: {e}")
        return None

# 使用gather的return_exceptions
results = await asyncio.gather(*tasks, return_exceptions=True)

# ❌ 坏的做法：不处理异常
results = await asyncio.gather(*tasks)  # 一个失败全部失败
```

#### 3. 避免阻塞操作

```python
# ✅ 好的做法：使用异步I/O
async with aiofiles.open("file.txt") as f:
    content = await f.read()

# ❌ 坏的做法：在异步函数中使用同步I/O
with open("file.txt") as f:  # 阻塞！
    content = f.read()
```

#### 4. 合理使用abatch

```python
# ✅ 好的做法：使用abatch批量处理
results = await model.abatch(inputs)

# ❌ 坏的做法：循环调用
results = []
for input in inputs:
    result = await model.ainvoke(input)
    results.append(result)
```

---

### 常见陷阱

#### 陷阱1：忘记await

```python
# ❌ 错误：忘记await
result = model.ainvoke("问题")  # 返回coroutine对象，不是结果

# ✅ 正确
result = await model.ainvoke("问题")
```

#### 陷阱2：在循环中串行执行

```python
# ❌ 错误：看起来异步，实际串行
results = []
for data in dataset:
    result = await model.ainvoke(data)  # 一个一个等待
    results.append(result)

# ✅ 正确：真正并发
tasks = [model.ainvoke(data) for data in dataset]
results = await asyncio.gather(*tasks)
```

#### 陷阱3：过度并发

```python
# ❌ 错误：并发太多
tasks = [model.ainvoke(data) for data in huge_dataset]  # 可能上千个
results = await asyncio.gather(*tasks)  # 触发API速率限制

# ✅ 正确：使用Semaphore控制
semaphore = asyncio.Semaphore(20)

async def controlled_call(data):
    async with semaphore:
        return await model.ainvoke(data)

tasks = [controlled_call(data) for data in huge_dataset]
results = await asyncio.gather(*tasks)
```

#### 陷阱4：混用同步和异步

```python
# ❌ 错误：在异步函数中调用同步方法
async def process():
    result = model.invoke("问题")  # 同步方法，会阻塞event loop
    return result

# ✅ 正确：全部使用异步
async def process():
    result = await model.ainvoke("问题")
    return result
```

---

### 性能优化技巧

#### 1. 分批处理大数据集

```python
# 处理1000个任务，分成50批，每批20个
async def process_in_batches(dataset, batch_size=20):
    results = []
    for i in range(0, len(dataset), batch_size):
        batch = dataset[i:i+batch_size]
        batch_results = await asyncio.gather(*[
            model.ainvoke(data) for data in batch
        ])
        results.extend(batch_results)
    return results
```

#### 2. 使用连接池

```python
# FastAPI自动管理连接池
from fastapi import FastAPI

app = FastAPI()

# 复用同一个model实例
@app.post("/chat")
async def chat(message: str):
    return await model.ainvoke(message)
```

#### 3. 预热连接

```python
# 在服务启动时预热
async def warm_up():
    await model.ainvoke("test")  # 建立连接

# FastAPI启动事件
@app.on_event("startup")
async def startup():
    await warm_up()
```

---

## 下一步学习

完成异步与并发学习后，建议学习：

1. **第20章：Error Handling & Retry** - 错误处理和重试策略
2. **生产部署** - 监控、日志、扩展
3. **高级优化** - 缓存、批处理、负载均衡

---

## 关键收获

✅ **异步适用于I/O密集型任务**：LLM API调用是典型场景

✅ **所有方法都有异步版本**：`ainvoke()`、`astream()`、`abatch()`

✅ **性能提升显著**：并发执行可获得3-10倍加速

✅ **使用asyncio.gather()并发执行**：自动管理任务

✅ **Semaphore控制并发数**：避免触发API限制

✅ **abatch()比循环ainvoke更高效**：自动优化

✅ **注意错误处理**：使用`return_exceptions=True`

✅ **避免常见陷阱**：忘记await、串行执行、过度并发、混用同步异步